In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [ ]:
university_towns.txt

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [7]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import re
    with open ('university_towns.txt') as file:
        lines=[]
        together=[]
        for line in file:
            line = re.sub(r'\(.*\)','',line)
            line = re.sub(r'\[.*\]','',line)
            lines.append(line.strip('\n '))
        for i in range(len(lines)):
            if lines[i] in states.values():
                stname = lines[i]
            else:
                together.append([stname,lines[i]])
    
    return pd.DataFrame(together, columns=["State", "RegionName"])
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [8]:
def university_towns():
    import re
    with open ('university_towns.txt') as file:
        lines=[]
        together=[]
        for line in file:
            line = re.sub(r'\(.*\)','',line)
            line = re.sub(r'\[.*\]','',line)
            lines.append(line.strip('\n '))
        for i in range(len(lines)):
            if lines[i] in states.values():
                stname = lines[i]
            else:
                together.append([stname,lines[i]])
    
    return together
university_towns()

[['Alabama', 'Auburn'],
 ['Alabama', 'Florence'],
 ['Alabama', 'Jacksonville'],
 ['Alabama', 'Livingston'],
 ['Alabama', 'Montevallo'],
 ['Alabama', 'Troy'],
 ['Alabama', 'Tuscaloosa'],
 ['Alabama', 'Tuskegee'],
 ['Alaska', 'Fairbanks'],
 ['Arizona', 'Flagstaff'],
 ['Arizona', 'Tempe'],
 ['Arizona', 'Tucson'],
 ['Arkansas', 'Arkadelphia'],
 ['Arkansas', 'Conway'],
 ['Arkansas', 'Fayetteville'],
 ['Arkansas', 'Jonesboro'],
 ['Arkansas', 'Magnolia'],
 ['Arkansas', 'Monticello'],
 ['Arkansas', 'Russellville'],
 ['Arkansas', 'Searcy'],
 ['California', 'Angwin'],
 ['California', 'Arcata'],
 ['California', 'Berkeley'],
 ['California', 'Chico'],
 ['California', 'Claremont'],
 ['California', 'Cotati'],
 ['California', 'Davis'],
 ['California', 'Irvine'],
 ['California', 'Isla Vista'],
 ['California', 'University Park, Los Angeles'],
 ['California', 'Merced'],
 ['California', 'Orange'],
 ['California', 'Palo Alto'],
 ['California', 'Pomona  and formerly Pomona College'],
 ['California', 'Redlan

In [17]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP=pd.read_excel('gdplev.xls',header=0)
    GDP=GDP.iloc[219:,:]
    n = len(GDP.iloc[:,6])
    for i in range(n-2):
        if (GDP.iloc[i,6] > GDP.iloc[i+1,6]) & (GDP.iloc[i+1,6] > GDP.iloc[i+2,6]):
            return GDP.iloc[i,4]
get_recession_start()

'2008q2'

In [16]:
GDP=pd.read_excel('gdplev.xls',header=0)
GDP=GDP.iloc[252:,:]
GDP

,"Current-Dollar and ""Real"" Gross Domestic Product",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2017-01-27 00:00:00
252,NaN,NaN,NaN,NaN,2008q2,14813,14963.4,NaN
253,NaN,NaN,NaN,NaN,2008q3,14843,14891.6,NaN
254,NaN,NaN,NaN,NaN,2008q4,14549.9,14577,NaN
255,NaN,NaN,NaN,NaN,2009q1,14383.9,14375,NaN
256,NaN,NaN,NaN,NaN,2009q2,14340.4,14355.6,NaN
257,NaN,NaN,NaN,NaN,2009q3,14384.1,14402.5,NaN
258,NaN,NaN,NaN,NaN,2009q4,14566.5,14541.9,NaN
259,NaN,NaN,NaN,NaN,2010q1,14681.1,14604.8,NaN
260,NaN,NaN,NaN,NaN,2010q2,14888.6,14745.9,NaN
261,NaN,NaN,NaN,NaN,2010q3,15057.7,14845.5,NaN


In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP=pd.read_excel('gdplev.xls',header=0)
    GDP=GDP.iloc[252:,:]
    n = len(GDP.iloc[:,6])
    for i in range(n-2):
        if (GDP.iloc[i,6] < GDP.iloc[i+1,6]) & (GDP.iloc[i+1,6] < GDP.iloc[i+2,6]):
            return GDP.iloc[i+1,4]
get_recession_end()

'2009q3'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP=pd.read_excel('gdplev.xls',header=0)
    GDP=GDP.iloc[252:,:]
    n = len(GDP.iloc[:,6])
    for i in range(n-2):
        if (GDP.iloc[i,6] < GDP.iloc[i+1,6]) & (GDP.iloc[i+1,6] < GDP.iloc[i+2,6]):
            return GDP.iloc[i,4]
get_recession_bottom()

'2009q2'

In [82]:
stname=[]

for i in range(len(house['State'])):
    stname.append(states[house['State'][i]])
house['State']=stname


,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,New York,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,California,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,Illinois,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,Pennsylvania,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,Arizona,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,18959,Las Vegas,Nevada,Las Vegas,Clark,6,121600.0,120900.0,120400.0,120300.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,54296,San Diego,California,San Diego,San Diego,7,161100.0,160700.0,160400.0,160100.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,38128,Dallas,Texas,Dallas-Fort Worth,Dallas,8,NaN,NaN,NaN,NaN,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,33839,San Jose,California,San Jose,Santa Clara,9,224500.0,224900.0,225400.0,226100.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,25290,Jacksonville,Florida,Jacksonville,Duval,10,77500.0,77200.0,76800.0,76600.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [109]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

    house=pd.read_csv('City_Zhvi_AllHomes.csv',header=0)
    stname=[]
    for i in range(len(house['State'])):
        stname.append(states[house['State'][i]])
    house['State']=stname
    time=house.iloc[:,51:]
    #print(np.mean(time.iloc[:,0:0+3],axis=1))
    a=[np.mean(time.ix[:,3*i:3*i+3],axis=1) for i in range(66)]
    a.append(np.mean(time.iloc[:,-2:],axis=1))
    name=[]
    for i in range(0,17):
        for j in range(1,5):
            if i>=10:
                name.append('20'+str(i)+'q'+str(j))
            else:
                name.append('200'+str(i)+'q'+str(j)) 
    name.pop()
    a=pd.DataFrame(a)
    a=a.T
    a.columns=name
    a=a.set_index([house['State'],house['RegionName']])
    
    return a
#convert_housing_data_to_quarters()

In [110]:
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [127]:
x=convert_housing_data_to_quarters().iloc[:100,:]
#x=pd.DataFrame(x)
x.head()
y=get_list_of_university_towns()
y.head()
x.merge(y,left_index=True, right_on=['State','RegionName'],how='inner')

,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3,State,RegionName
252,132600.000000,134366.666667,135400.000000,137000.000000,139533.333333,141733.333333,143366.666667,146133.333333,149333.333333,150933.333333,...,175466.666667,177500.000000,181600.000000,186766.666667,190633.333333,194600.000000,197200.000000,199950.0,Nevada,Las Vegas
38,222900.000000,234366.666667,245433.333333,256033.333333,267200.000000,276266.666667,284500.000000,291933.333333,301233.333333,312866.666667,...,496433.333333,503366.666667,512066.666667,519766.666667,525466.666667,529333.333333,536233.333333,539750.0,California,San Diego
447,84466.666667,83866.666667,84866.666667,87833.333333,89733.333333,89300.000000,89066.666667,90900.000000,92566.666667,93800.000000,...,111533.333333,113700.000000,121133.333333,128566.666667,134600.000000,140500.000000,144600.000000,149300.0,Texas,Dallas
442,142966.666667,145266.666667,149466.666667,155733.333333,161233.333333,160733.333333,159533.333333,160033.333333,158966.666667,157500.000000,...,258133.333333,266500.000000,275033.333333,281633.333333,287233.333333,293500.000000,301433.333333,304450.0,Texas,Austin
335,94366.666667,95833.333333,97133.333333,98266.666667,99400.000000,100266.666667,101066.666667,102200.000000,103400.000000,104800.000000,...,106433.333333,107866.666667,109433.333333,111566.666667,115000.000000,116700.000000,118200.000000,120100.0,Ohio,Columbus
436,72500.000000,73200.000000,73866.666667,74000.000000,74166.666667,74933.333333,75500.000000,76066.666667,76333.333333,76766.666667,...,71166.666667,70533.333333,68700.000000,68666.666667,69533.333333,70900.000000,74166.666667,75900.0,Tennessee,Memphis
175,206933.333333,219166.666667,233100.000000,242500.000000,249600.000000,257066.666667,266933.333333,274966.666667,282500.000000,289300.000000,...,440733.333333,448500.000000,455366.666667,463966.666667,471633.333333,482600.000000,490366.666667,501700.0,Massachusetts,Boston
438,113833.333333,115266.666667,115866.666667,116933.333333,118033.333333,119166.666667,120100.000000,120800.000000,121566.666667,122633.333333,...,166900.000000,171466.666667,176266.666667,181800.000000,189200.000000,195066.666667,200366.666667,206100.0,Tennessee,Nashville
506,78033.333333,79066.666667,81033.333333,82333.333333,84033.333333,85566.666667,87066.666667,88400.000000,89533.333333,91366.666667,...,91966.666667,93333.333333,94100.000000,94133.333333,94566.666667,94666.666667,96366.666667,98850.0,Wisconsin,Milwaukee
11,101833.333333,102966.666667,104466.666667,105666.666667,107200.000000,108766.666667,110566.666667,112800.000000,115000.000000,117200.000000,...,144233.333333,144166.666667,145133.333333,146600.000000,148166.666667,149533.333333,151166.666667,152700.0,Arizona,Tucson


In [141]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    newdata=convert_housing_data_to_quarters()
    y=get_list_of_university_towns()
    newdata['Test']=newdata['2009q2']-newdata['2008q2']
    university=newdata.merge(y,left_index=True, right_on=['State','RegionName'],how='inner').dropna()
    out=newdata.merge(y,left_index=True, right_on=['State','RegionName'],how='outer').dropna()

    #out=out['Test']
    ttest_ind(university['Test'], out['Test'])
    if ttest_ind(university['Test'], out['Test'])[1]<0.01:
        different=True
    else:
        different=False
    p=ttest_ind(university['Test'], out['Test'])[1]
    if np.mean(university['Test'])<np.mean(out['Test']):
        better="university town"
    else:
        better="non-university town"
    return tuple([different, p, better])
run_ttest()

(False, 0.012179201131033959, 'non-university town')